In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import paired_distances
from sklearn.metrics import jaccard_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import re

In [24]:
df_2010s = pd.read_pickle('../2010s_albums.pkl')
df_2010s.head(3)

,name,album,year,artist,featured_artists,track_number,tracks_on_album,explicit,duration_ms,popularity,...,album_label,artist_spotify_link,followers,artist_genres,artist_image_url,artist_popularity,sections,tatums,beats,bars
0,Wesley's Theory,To Pimp A Butterfly,2015,Kendrick Lamar,Thundercat,1,16,1,287360,63,...,Aftermath,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,17080939,"[conscious hip hop, hip hop, rap, west coast rap]",https://i.scdn.co/image/3a836196bfb341f736c7fe...,89,"[{'start': 0.0, 'duration': 16.55891, 'confide...","[{'start': 0.56645, 'duration': 0.29416, 'conf...","[{'start': 0.56645, 'duration': 0.58832, 'conf...","[{'start': 0.56645, 'duration': 2.20908, 'conf..."
1,For Free? - Interlude,To Pimp A Butterfly,2015,Kendrick Lamar,NaN,2,16,1,130893,57,...,Aftermath,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,17080939,"[conscious hip hop, hip hop, rap, west coast rap]",https://i.scdn.co/image/3a836196bfb341f736c7fe...,89,"[{'start': 0.0, 'duration': 4.9025, 'confidenc...","[{'start': 0.67226, 'duration': 0.32529, 'conf...","[{'start': 0.67226, 'duration': 0.48793, 'conf...","[{'start': 0.67226, 'duration': 1.89954, 'conf..."
2,King Kunta,To Pimp A Butterfly,2015,Kendrick Lamar,NaN,3,16,1,234693,74,...,Aftermath,https://open.spotify.com/artist/2YZyLoL8N0Wb9x...,17080939,"[conscious hip hop, hip hop, rap, west coast rap]",https://i.scdn.co/image/3a836196bfb341f736c7fe...,89,"[{'start': 0.0, 'duration': 6.24966, 'confiden...","[{'start': 0.08182, 'duration': 0.2886, 'confi...","[{'start': 0.08182, 'duration': 0.5772, 'confi...","[{'start': 1.2252, 'duration': 2.22486, 'confi..."


In [25]:
df_2010s.columns

Index(['name', 'album', 'year', 'artist', 'featured_artists', 'track_number',
       'tracks_on_album', 'explicit', 'duration_ms', 'popularity',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'time_signature',
       'artist_uri', 'album_uri', 'release_date', 'album_image_url',
       'track_length', 'tempo_confidence', 'end_fade_in', 'start_fade_out',
       'end_silence_time', 'album_label', 'artist_spotify_link', 'followers',
       'artist_genres', 'artist_image_url', 'artist_popularity', 'sections',
       'tatums', 'beats', 'bars'],
      dtype='object')

In [26]:
df_2010_str, df_2010_ref, df_2010_num, df_2010_arrs = split_df_columns(df_2010s)
df_2010_str

artist  \
album               name                                           
To Pimp A Butterfly Wesley's Theory               Kendrick Lamar   
                    For Free? - Interlude         Kendrick Lamar   
                    King Kunta                    Kendrick Lamar   
                    Institutionalized             Kendrick Lamar   
                    These Walls                   Kendrick Lamar   
...                                                          ...   
Returnal            Returnal               Oneohtrix Point Never   
                    Pelham Island Road     Oneohtrix Point Never   
                    Where Does Time Go     Oneohtrix Point Never   
                    Ouroboros              Oneohtrix Point Never   
                    Preyouandi             Oneohtrix Point Never   

                                          featured_artists release_date  \
album               name                                                  
To Pimp A Butterfly Wesley's Theory             Thundercat   2015-03-16   
                    For Free? - Interlude      No Features   2015-03-16   
                    King Kunta                 No Features   2015-03-16   
                    Institutionalized           Snoop Dogg   2015-03-16   
                    These Walls                 Thundercat   2015-03-16   
...                                                    ...          ...   
Returnal            Returnal                   No Features   2010-06-21   
                    Pelham Island Road         No Features   2010-06-21   
                    Where Does Time Go         No Features   2010-06-21   
                    Ouroboros                  No Features   2010-06-21   
                    Preyouandi                 No Features   2010-06-21   

                                             album_label  \
album               name                                   
To Pimp A Butterfly Wesley's Theory            Aftermath   
                    For Free? - Interlude      Aftermath   
                    King Kunta                 Aftermath   
                    Institutionalized          Aftermath   
                    These Walls                Aftermath   
...                                                  ...   
Returnal            Returnal               Editions Mego   
                    Pelham Island Road     Editions Mego   
                    Where Does Time Go     Editions Mego   
                    Ouroboros              Editions Mego   
                    Preyouandi             Editions Mego   

                                                                               artist_genres  
album               name                                                                      
To Pimp A Butterfly Wesley's Theory        [conscious hip hop, hip hop, rap, west coast rap]  
                    For Free? - Interlude  [conscious hip hop, hip hop, rap, west coast rap]  
                    King Kunta             [conscious hip hop, hip hop, rap, west coast rap]  
                    Institutionalized      [conscious hip hop, hip hop, rap, west coast rap]  
                    These Walls            [conscious hip hop, hip hop, rap, west coast rap]  
...                                                                                      ...  
Returnal            Returnal               [ambient, art pop, chamber psych, chillwave, d...  
                    Pelham Island Road     [ambient, art pop, chamber psych, chillwave, d...  
                    Where Does Time Go     [ambient, art pop, chamber psych, chillwave, d...  
                    Ouroboros              [ambient, art pop, chamber psych, chillwave, d...  
                    Preyouandi             [ambient, art pop, chamber psych, chillwave, d...  

[10167 rows x 5 columns]

In [27]:
df_2010_str.columns

Index(['artist', 'featured_artists', 'release_date', 'album_label',
       'artist_genres'],
      dtype='object')

In [22]:
df_2010_str['bag_of_words'] = ''
columns = df_2010_str.columns
for index, row in df_2010_str.iterrows():
    words = ''
    for col in columns:
        if col == 'artist_genres':
            words += ' '.join(row[col])+ ' '
        else:
            words += row[col]+ ' '
    row['bag_of_words'] = words
    
df_2010_str.drop(columns = [col for col in df_2010_str.columns if col!= 'bag_of_words'], inplace = True)

df_2010_str

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


bag_of_words
album               name                              
To Pimp A Butterfly Wesley's Theory                   
                    For Free? - Interlude             
                    King Kunta                        
                    Institutionalized                 
                    These Walls                       
...                                                ...
Returnal            Returnal                          
                    Pelham Island Road                
                    Where Does Time Go                
                    Ouroboros                         
                    Preyouandi                        

[10167 rows x 1 columns]

In [2]:
df = pd.read_csv('../my_albums.csv', engine='python')

In [3]:
df.set_index(['album','name'], inplace=True)

In [4]:
df

year  \
album                       name                                                       
The Saga Continues          Wu-Tang The Saga Continues Intro (feat. RZA)        2017   
                            Lesson Learn'd (feat. Inspectah Deck and Redman)    2017   
                            Fast and Furious (feat. Hue Hef and Raekwon)        2017   
                            Famous Fighters (Skit)                              2017   
                            If Time Is Money (Fly Navigation) [feat. Method...  2017   
...                                                                              ...   
Modern Vampires of the City Finger Back                                         2013   
                            Worship You                                         2013   
                            Ya Hey                                              2013   
                            Hudson                                              2013   
                            Young Lion                                          2013   

                                                                                         artist  \
album                       name                                                                  
The Saga Continues          Wu-Tang The Saga Continues Intro (feat. RZA)           Wu-Tang Clan   
                            Lesson Learn'd (feat. Inspectah Deck and Redman)       Wu-Tang Clan   
                            Fast and Furious (feat. Hue Hef and Raekwon)           Wu-Tang Clan   
                            Famous Fighters (Skit)                                 Wu-Tang Clan   
                            If Time Is Money (Fly Navigation) [feat. Method...     Wu-Tang Clan   
...                                                                                         ...   
Modern Vampires of the City Finger Back                                         Vampire Weekend   
                            Worship You                                         Vampire Weekend   
                            Ya Hey                                              Vampire Weekend   
                            Hudson                                              Vampire Weekend   
                            Young Lion                                          Vampire Weekend   

                                                                               featured_artists  \
album                       name                                                                  
The Saga Continues          Wu-Tang The Saga Continues Intro (feat. RZA)                    RZA   
                            Lesson Learn'd (feat. Inspectah Deck and Redman)             Redman   
                            Fast and Furious (feat. Hue Hef and Raekwon)                Raekwon   
                            Famous Fighters (Skit)                                          NaN   
                            If Time Is Money (Fly Navigation) [feat. Method...       Method Man   
...                                                                                         ...   
Modern Vampires of the City Finger Back                                                     NaN   
                            Worship You                                                     NaN   
                            Ya Hey                                                          NaN   
                            Hudson                                                          NaN   
                            Young Lion                                                      NaN   

                                                                                track_number  \
album                       name                                                               
The Saga Continues          Wu-Tang The Saga Continues Intro (feat. RZA)                   1   
                            Lesson Learn'd (feat. Inspectah Deck and Re

In [5]:
df['has_featured_artist'] = np.where(df['featured_artists'].isna(), 0, 1)
df.head()

year  \
album              name                                                       
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)        2017   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)    2017   
                   Fast and Furious (feat. Hue Hef and Raekwon)        2017   
                   Famous Fighters (Skit)                              2017   
                   If Time Is Money (Fly Navigation) [feat. Method...  2017   

                                                                             artist  \
album              name                                                               
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)        Wu-Tang Clan   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)    Wu-Tang Clan   
                   Fast and Furious (feat. Hue Hef and Raekwon)        Wu-Tang Clan   
                   Famous Fighters (Skit)                              Wu-Tang Clan   
                   If Time Is Money (Fly Navigation) [feat. Method...  Wu-Tang Clan   

                                                                      featured_artists  \
album              name                                                                  
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)                    RZA   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)             Redman   
                   Fast and Furious (feat. Hue Hef and Raekwon)                Raekwon   
                   Famous Fighters (Skit)                                          NaN   
                   If Time Is Money (Fly Navigation) [feat. Method...       Method Man   

                                                                       track_number  \
album              name                                                               
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)                   1   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)               2   
                   Fast and Furious (feat. Hue Hef and Raekwon)                   3   
                   Famous Fighters (Skit)                                         4   
                   If Time Is Money (Fly Navigation) [feat. Method...             5   

                                                                       tracks_on_album  \
album              name                                                                  
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)                     18   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)                 18   
                   Fast and Furious (feat. Hue Hef and Raekwon)                     18   
                   Famous Fighters (Skit)                                           18   
                   If Time Is Money (Fly Navigation) [feat. Method...               18   

                                                                       explicit  \
album              name                                                           
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)               1   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)           1   
                   Fast and Furious (feat. Hue Hef and Raekwon)               1   
                   Famous Fighters (Skit)                                     1   
                   If Time Is Money (Fly Navigation) [feat. Method...         1   

                                                                       duration_ms  \
album              name                                                              
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)              91080   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)         202667   
                   Fast and Furious (feat. Hue Hef and Raekwon)             222311   
                   Famo

In [6]:
df.columns

Index(['year', 'artist', 'featured_artists', 'track_number', 'tracks_on_album',
       'explicit', 'duration_ms', 'popularity', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri',
       'track_href', 'analysis_url', 'time_signature', 'artist_uri',
       'album_uri', 'release_date', 'album_image_url', 'track_length',
       'tempo_confidence', 'end_fade_in', 'start_fade_out', 'end_silence_time',
       'sections', 'tatums', 'beats', 'bars', 'has_featured_artist'],
      dtype='object')

## Seperating Columns based on dtype, also removing a few that don't seem important

In [7]:
df_str = df[['artist', 'featured_artists', 'release_date']]

df_ref = df[['id', 'uri', 'track_href', 'analysis_url',
          'artist_uri', 'album_uri', 'album_image_url']]

df_num = df[['year', 'has_featured_artist', 'track_number', 'tracks_on_album', 'explicit', 'duration_ms', 'popularity',
          'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
          'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
          'track_length', 'tempo_confidence', 'end_fade_in', 'start_fade_out','end_silence_time']]

df_arrs = df[['sections', 'tatums', 'beats', 'bars']]

### Looking at string values

In [8]:
df_ref.head()

id  \
album              name                                                                         
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)        2EsC5nxXhoMrOsYbt59VUM   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)    2RQ47bAanXMoCKIIUn5Kxn   
                   Fast and Furious (feat. Hue Hef and Raekwon)        2B6lQV5InRy13vd7wl26jg   
                   Famous Fighters (Skit)                              09zHR2ttyr84ibzpBVrmNu   
                   If Time Is Money (Fly Navigation) [feat. Method...  5l3C7CgpiOoGF6B8qzbOR3   

                                                                                                        uri  \
album              name                                                                                       
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)        spotify:track:2EsC5nxXhoMrOsYbt59VUM   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)    spotify:track:2RQ47bAanXMoCKIIUn5Kxn   
                   Fast and Furious (feat. Hue Hef and Raekwon)        spotify:track:2B6lQV5InRy13vd7wl26jg   
                   Famous Fighters (Skit)                              spotify:track:09zHR2ttyr84ibzpBVrmNu   
                   If Time Is Money (Fly Navigation) [feat. Method...  spotify:track:5l3C7CgpiOoGF6B8qzbOR3   

                                                                                                              track_href  \
album              name                                                                                                    
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)        https://api.spotify.com/v1/tracks/2EsC5nxXhoMr...   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)    https://api.spotify.com/v1/tracks/2RQ47bAanXMo...   
                   Fast and Furious (feat. Hue Hef and Raekwon)        https://api.spotify.com/v1/tracks/2B6lQV5InRy1...   
                   Famous Fighters (Skit)                              https://api.spotify.com/v1/tracks/09zHR2ttyr84...   
                   If Time Is Money (Fly Navigation) [feat. Method...  https://api.spotify.com/v1/tracks/5l3C7CgpiOoG...   

                                                                                                            analysis_url  \
album              name                                                                                                    
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)        https://api.spotify.com/v1/audio-analysis/2EsC...   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)    https://api.spotify.com/v1/audio-analysis/2RQ4...   
                   Fast and Furious (feat. Hue Hef and Raekwon)        https://api.spotify.com/v1/audio-analysis/2B6l...   
                   Famous Fighters (Skit)                              https://api.spotify.com/v1/audio-analysis/09zH...   
                   If Time Is Money (Fly Navigation) [feat. Method...  https://api.spotify.com/v1/audio-analysis/5l3C...   

                                                                                                  artist_uri  \
album              name                                                                                        
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)        spotify:artist:34EP7KEpOjXcM2TCat1ISk   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)    spotify:artist:34EP7KEpOjXcM2TCat1ISk   
                   Fast and Furious (feat. Hue Hef and Raekwon)        spotify:artist:34EP7KEpOjXcM2TCat1ISk   
                   Famous Fighters (Skit)                              spotify:artist:34EP7KEpOjXcM2TCat1ISk   
                   If Time Is Money (Fly Navigation) [feat. Method...  spotify:artist:34EP7KEpOjXcM2TCat1ISk   

                                                                                                  album_uri  \

In [9]:
df_str.fillna('No Features', inplace=True)
df_str['release_date'] =  pd.to_datetime(df_str['release_date'], format='%Y-%m-%d')

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/pandas/core/frame.py:4465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [10]:
df_str.head()

artist  \
album              name                                                               
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)        Wu-Tang Clan   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)    Wu-Tang Clan   
                   Fast and Furious (feat. Hue Hef and Raekwon)        Wu-Tang Clan   
                   Famous Fighters (Skit)                              Wu-Tang Clan   
                   If Time Is Money (Fly Navigation) [feat. Method...  Wu-Tang Clan   

                                                                      featured_artists  \
album              name                                                                  
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)                    RZA   
                   Lesson Learn'd (feat. Inspectah Deck and Redman)             Redman   
                   Fast and Furious (feat. Hue Hef and Raekwon)                Raekwon   
                   Famous Fighters (Skit)                                  No Features   
                   If Time Is Money (Fly Navigation) [feat. Method...       Method Man   

                                                                      release_date  
album              name                                                             
The Saga Continues Wu-Tang The Saga Continues Intro (feat. RZA)         2017-10-13  
                   Lesson Learn'd (feat. Inspectah Deck and Redman)     2017-10-13  
                   Fast and Furious (feat. Hue Hef and Raekwon)         2017-10-13  
                   Famous Fighters (Skit)                               2017-10-13  
                   If Time Is Money (Fly Navigation) [feat. Method...   2017-10-13

### Looking at numerical values

In [11]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1109 entries, ('The Saga Continues', 'Wu-Tang The Saga Continues Intro (feat. RZA)') to ('Modern Vampires of the City', 'Young Lion')
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 1109 non-null   int64  
 1   has_featured_artist  1109 non-null   int64  
 2   track_number         1109 non-null   int64  
 3   tracks_on_album      1109 non-null   int64  
 4   explicit             1109 non-null   int64  
 5   duration_ms          1109 non-null   int64  
 6   popularity           1109 non-null   int64  
 7   danceability         1109 non-null   float64
 8   energy               1109 non-null   float64
 9   key                  1109 non-null   int64  
 10  loudness             1109 non-null   float64
 11  mode                 1109 non-null   int64  
 12  speechiness          1109 non-null   float64
 13  acousticness         1109 no

In [12]:
df_num.head(1).T

album,The Saga Continues
name,Wu-Tang The Saga Continues Intro (feat. RZA)
year,2017.00000
has_featured_artist,1.00000
track_number,1.00000
tracks_on_album,18.00000
explicit,1.00000
duration_ms,91080.00000
popularity,32.00000
danceability,0.48000
energy,0.71800


### Looking at the Audio Analysis values

In [ ]:
ex_1 = df_arrs['bars'].head(1)
ex_1.values

In [ ]:
vals = ex_1.to_dict()

In [ ]:
type(vals)

In [ ]:
pd.DataFrame(vals, index=['1'])

In [ ]:
from collections import OrderedDict, defaultdict
dd = defaultdict(list)
ex1_vals = ex_1.to_dict(dd)
ex1_vals

In [ ]:
ex1_vals.values()

In [ ]:
ex1_vals_lst = ex1_vals.tolist()
ex1_vals_lst[0]

In [ ]:
type(ex1_vals_lst[0])

In [ ]:
for x in ex1_vals.tolist():
    print(x + '\n')

In [ ]:
class ItemRecommender():
    '''
    Content based item recommender
    '''
    def __init__(self, similarity_measure=cosine_similarity):
        self.similarity_matrix = None
        self.item_names = None
        self.similarity_measure = similarity_measure

    
    def fit(self, X, titles=None):
        '''
        Takes a numpy array of the item attributes and creates the similarity matrix

        INPUT -
            X: NUMPY ARRAY - Rows are items, columns are feature values / or DF
            titles: LIST - List of the item names/titles in order of the numpy arrray
        
        OUTPUT - None


        Notes:  You might want to keep titles and X as attributes to refer to them later

        Create a similarity matrix of item to item similarity
        '''

        # While keeping this as a sparse matrix would be best the cosign sim
        # function returns a array so there is no reason.
        
        if isinstance(X, pd.DataFrame):
            self.item_counts = X
            self.item_names = X.index
            self.similarity_df = pd.DataFrame(self.similarity_measure(X.values, X.values),
                 index = self.item_names)
        else:
            self.item_counts = X
            self.similarity_df = pd.DataFrame(self.similarity_measure(X, X),
                 index = titles)
            self.item_names = self.similarity_df.index

        
    def get_recommendations(self, item, n=5):
        '''
        Returns the top n items related to the item passed in
        INPUT:
            item    - STRING - Name of item in the original DataFrame 
            n       - INT    - Number of top related items to return 
        OUTPUT:
            items - List of the top n related item names

        For a given item find the n most similar items to it (this can be done using the similarity matrix created in the fit method)
        '''
        return self.item_names[self.similarity_df.loc[item].values.argsort()[-(n+1):-1]].values[::-1]


    def get_user_profile(self, items):
        '''
        Takes a list of items and returns a user profile. A vector representing the likes of the user.
        INPUT: 
            items  -   LIST - list of movie names user likes / has seen

        OUTPUT: 
            user_profile - NP ARRAY - array representing the likes of the user 
                    The columns of this will match the columns of the trained on matrix
    

        Using the list of items liked by the user create a profile which will be a 1 x number of features array.  This should be the addition of the values for all liked item features (you can choose how to normalize if you think it is needed)
        '''
        user_profile = np.zeros(self.item_counts.shape[1])
        for item in items:
            user_profile += self.item_counts.loc[item].values

        return user_profile


    def get_user_recommendation(self, items, n=5):
        '''
        Takes a list of movies user liked and returns the top n items for that user

        INPUT 
            items  -   LIST - list of movie names user likes / has seen
            n -  INT - number of items to return

        OUTPUT 
            items - LIST - n recommended items

    
        Make use of the get_user_profile method to create a user profile that will be used to get the similarity to all items and recommend the top n.
        '''
        num_items = len(items)
        user_profile = self.get_user_profile(items)

        user_sim =  self.similarity_measure(self.item_counts, user_profile.reshape(1,-1))

        return self.item_names[user_sim[:,0].argsort()[-(num_items+n):-num_items]].values[::-1]





